## Minimax
The minimax algorythm can be very useful when deciding next moves, though it is only a loose fit in the category of "machine learning". This is because minimax is not trained and does not learn. Instead, it acts more like we traditionally assume computers would work: it computes every possibility and chooses the best one. "Best" is a little to vague for us, so let's define it a bit more. Minimax stands for maximizing the minimum gain. This means the algorythm will always "play it safe" and choose the least risky option. So if the two choices were: "maybe lose 3 points or maybe gain 10" and "maybe lose 2 points or maybe gain 1" it will allways take the latter. Now this may not seem particularly helpful, but there are many situations where it works very well. One of those is Tic-Tac-Toe, which you can look at in the "examples" folder. First, here is a great video explaining how the algorythm works in detail. Feel free to pause and re-watch as often as needed to understand.

[![Minimax - Sebastian Lague](https://img.youtube.com/vi/l-hh51ncgDI/0.jpg)](https://www.youtube.com/watch?v=l-hh51ncgDI)

Here is the algorythm from the video written in python (note `state` functions are not implemented):

In [ ]:
import numpy as np

In [ ]:
def minimax(state, depth, alpha, beta, maximizing_player):
    if depth == 0 or state.is_game_over():
        return state.evaluation()

    if maximizing_player:
        max_eval = -np.Infinity
        for next_state in state.next_possible_states():
            evaluation = minimax(next_state, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = np.Infinity
        for next_state in state.next_possible_states():
            evaluation = minimax(next_state, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, evaluation)
            alpha = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval

Depending on what you are utilizing the algorythm for, it will likely need some tweaking. For basic Tic-Tac-Toe, a game with no intermediate values, it will look something like this:

In [ ]:
def minimax(state, depth, alpha, beta, maximizing_player):
    #returns either 1, -1, or 0 for no winner
    winner = state.winner()
    #returns if max depth reached, someone has won, or there are no moves left
    if depth == 0 or winner != 0 or len(state.next_possible_states()) == 0:
        return winner

    if maximizing_player:
        max_eval = -np.Infinity
        for next_state in state.next_possible_states():
            evaluation = minimax(next_state, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = np.Infinity
        for next_state in state.next_possible_states():
            evaluation = minimax(next_state, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, evaluation)
            alpha = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval

Currently, this version of minimax treats all wins equally. However, it would be better if it preferred quicker wins. As such, let's add a decay that lowers the value every move, causing it to prefer quicker wins. This also ends up making it lose slower, allowing more opportunities for the opposing player to make an error.

In [ ]:
decay_rate = .9
def minimax(state, depth, alpha, beta, maximizing_player):
    #returns either 1, -1, or 0 for no winner
    winner = state.winner()
    #returns if max depth reached, someone has won, or there are no moves left
    if depth == 0 or winner != 0 or len(state.next_possible_states()) == 0:
        return winner

    if maximizing_player:
        max_eval = -np.Infinity
        for next_state in state.next_possible_states():
            #loses 10% of it's value for every move required
            evaluation = decay_rate * minimax(next_state, depth - 1, alpha, beta, False)
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = np.Infinity
        for next_state in state.next_possible_states():
            #loses 10% of it's value for every move required
            evaluation = decay_rate * minimax(next_state, depth - 1, alpha, beta, True)
            min_eval = min(min_eval, evaluation)
            alpha = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval

The above code is still not that great, as it assumes the opponent will play perfectly. Therefore, will not consider plays that might cause a win as it assumes the opponent will see them. In Tic-Tac-Toe, many of the possible moves will lead to a tie and therefore have the same apparent value to minimax. We can make this smarter by including a bias, meaning that even if all options are equal in perfect play, the one with the most possibilities for a win gets chosen. We can do this by also adding a small amount to the output which will only matter when the main minimax values are equal.

In [ ]:
decay_rate = .9
bias_multiplier = .01
def minimax(state, depth, alpha, beta, maximizing_player):
    #returns either 1, -1, or 0 for no winner
    winner = state.winner()
    #initialize bias to 0
    bias = 0
    #returns if max depth reached, someone has won, or there are no moves left
    if depth == 0 or winner != 0 or len(state.next_possible_states()) == 0:
        if winner!= 0:
            bias += winner * bias_multiplier
        return winner, bias

    if maximizing_player:
        max_eval = -np.Infinity
        for next_state in state.next_possible_states():
            evaluation, delta_bias = minimax(next_state, depth - 1, alpha, beta, False)
            #loses 10% of it's value for every move required
            evaluation *= decay_rate
            #update this moves bias based on next moves' bias
            bias += delta_bias
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval, bias

    else:
        min_eval = np.Infinity
        for next_state in state.next_possible_states():
            evaluation, delta_bias = minimax(next_state, depth - 1, alpha, beta, True)
            #loses 10% of it's value for every move required
            evaluation *= decay_rate
            #update this moves bias based on next moves' bias
            bias += delta_bias
            min_eval = min(min_eval, evaluation)
            alpha = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval, bias

We now have a algorythm which will work well with Tic-Tac-Toe like games. While the original algorythm is best suited for games with live scores, it can be easily modified to suit your needs. Hopefully this gave you some ideas about how to use minimax in your own projects, and how to approach modifying it to better suit your needs. Check out the "examples" directory for a simple minimax Tic-Tac-Toe implementation as well as a more complex one using it in conjunction with reinforcement learning.